In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchmark").getOrCreate()


# UDF
def udf_upper(text):
    return text.upper()


spark.udf.register("udf_upper", udf_upper)

In [ ]:
df = spark.read.parquet(f"large_string_5row.parquet")
df.createOrReplaceTempView("T")
df.printSchema()
print("n_row:", df.count(), "n_col:", len(df.columns))
# df.show(1)

In [ ]:
# builtin: 50m_string on large_string_5row.parquet
# builtin: 40m_string on large_string_10row.parquet
spark.sql("SELECT upper(50m_string) FROM T").write.parquet("output_1.parquet", mode="overwrite")

In [ ]:
# UDF: 30m_string on large_string_5row.parquet
# UDF: 20m_string on large_string_10row.parquet
spark.sql("SELECT udf_upper(30m_string) FROM T").write.parquet("output_2.parquet", mode="overwrite")

# check

In [ ]:
# check
df_output = spark.read.parquet("output_2.parquet")
df_output.show(1)

In [5]:
spark.stop()